# **kéo dữ liệu từ id node về id way**

In [ ]:
#!pip install goto-statement
import numpy as np
import pandas as pd
from collections import Counter
from goto import with_goto

## Load data

In [ ]:
data = pd.read_csv('../data/before_nearest_new.csv')

# df = pd.DataFrame (data, columns = ['vehicle','point'])
data

In [ ]:
df_osm_way = pd.read_csv('../data/osm_way.csv').set_index('id')
df_osm_way

## Tiền xử lý

In [ ]:
def Conv_str_2_list(df_osm_way):
    for index in df_osm_way.index:
        df_osm_way.at[index, 'ref'] = df_osm_way['ref'][index].replace("[","" )
        df_osm_way.at[index, 'ref'] = df_osm_way['ref'][index].replace("]","" )
        df_osm_way.at[index, 'ref'] = list(df_osm_way['ref'][index].split(", "))
    return df_osm_way

In [ ]:
# def find_list_row(list_node, df_osm_way):
#     list_id_way=[]
    
#     # duyệt từng node xủa xe
#     for check_node in range(1,len(list_node)): 
        
#         #duyệt từng đường trong danh sách đường
#         for index in df_osm_way.index:
#             way_pass_node=[]
             
#              # xét mỗi node trong đường nếu trùng với node của xe thì cho vào queue
#             if (list_node[check_node] in df_osm_way['ref'][index] and list_node[check_node -1] in df_osm_way['ref'][index]) or ():
#                 list_id_way.append(df_osm_way['id'][index])
#                 break
        
#     list_id_way = list(dict.fromkeys(list_id_way))
    
#     return list_id_way

## hàm tìm kiếm id way từ id node

In [ ]:
# hàm tìm danh sách các id way từ list id node 
@with_goto
def find_list_row(list_node, df_osm_way):
    """
    input:
        list_node: list các id node
        df_osm_way: data way
    output: list các id way được lấy sau khi xét từng cặp node liền kề nhau từ list_node
    """
    
    list_id_way=[]
    way_pass_node_before = pd.DataFrame()
    
    # lấy danh sách way có chứa node đầu tiên
    for index in df_osm_way.index:
             # xét mỗi node trong đường nếu trùng với node của xe thì cho vào queuè
            if list_node[0] in df_osm_way['ref'][index]:
                way_pass_node_before = way_pass_node_before.append(df_osm_way.loc[index])
                
    # duyệt từng node xủa xe từ node thứ 2
    for check_node in range(1,len(list_node)):  
        way_pass_node= pd.DataFrame() # lưu trữ các data way có chứa node đang xét
        same_way = False
        
        #duyệt từng đường trong danh sách đường
        for index in df_osm_way.index:
             # xét mỗi node trong đường nếu trùng với node của xe 
            if list_node[check_node] in df_osm_way['ref'][index]:
                # nếu node trước đó trùng id way với node đang xét thì lấy id way
                if list_node[check_node - 1] in df_osm_way['ref'][index]:
                    list_id_way.append(index)
                    same_way = True
                way_pass_node = way_pass_node.append(df_osm_way.loc[index])   

        # kiểm tra 2 node có chung id way không
        if same_way:
            goto .next
        # xét tên đường trùng nhau với node trước đó
        for index_list_way in way_pass_node.index:
            if way_pass_node.loc[index_list_way]['name'] in way_pass_node_before['name'].values:
                list_id_way.append(index_list_way)
                break
                
        label .next
        way_pass_node_before = way_pass_node.copy()
        #print(way_pass_node_before)
        
    list_id_way = list(dict.fromkeys(list_id_way))
    
    return list_id_way

In [ ]:
# lấy danh sách id way từ dữ liệu node của các xe
def get_list_way(data, df_osm_way):
    list_route=[]
    df = data.copy()
    for vehicle in df:
        list_way =[]
        df.at[0,vehicle] = df.at[0,vehicle].replace("[","" )
        df.at[0,vehicle] = df.at[0,vehicle].replace("]","" )
        df.at[0,vehicle] = list(df.at[0,vehicle].split(", "))
    
        list_route.append(find_list_row(df.at[0,vehicle], df_osm_way))
        print(vehicle, list_route[-1])
    data_leght=len(df)
    df.loc[data_leght] = list_route
    return df

In [ ]:
%%time
df =get_list_way(data, df_osm_way)


## Save data

In [ ]:
df.to_csv("../data/list_way_vehicle_2.csv", index = False)   